# Running Best Model (Logistic Regression) On Unseen Test Data

## imports

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install dmba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 48.8 MB/s eta 0:00:00


In [ ]:
from dmba import classificationSummary

Colab environment detected.


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

In [ ]:
test = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn_TESTDATA.csv')

## Preprocessing

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [ ]:
test.Churn.value_counts()

,count
Churn,
No,5174
Yes,1869


In [ ]:
pd.set_option('display.max_columns', len(test.columns)+1)

In [ ]:
test.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
test.shape

(7043, 20)

In [ ]:
test.columns = [col[:1].lower()+col[1:] for col in test.columns]

In [ ]:
test = test.set_index('customerID')

### Importing traing+validation set to drop duplicates from test set

In [ ]:
original = pd.read_excel('/content/for_machine_learningdf.xlsx', index_col='customerID')

In [ ]:
original.shape

(5042, 31)

In [ ]:
to_drop = [i for i in test.index if i in original.index]

In [ ]:
test = test.drop(to_drop, axis=0)

In [ ]:
test.shape

(2002, 20)

**Test now ONLY has new. Continue preprocessing here when return. **

In [ ]:
test.drop('partner', axis=1, inplace=True)

In [ ]:
test = test.replace('No',False).replace('Yes',True)

<ipython-input-158-073043a4ff43>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace('No',False).replace('Yes',True)


In [ ]:
test_response = test.pop('churn')

In [ ]:
test.apply(lambda x: 1 in x)

,0
gender,False
seniorCitizen,False
dependents,False
tenure,False
phoneService,False
multipleLines,False
internetService,False
onlineSecurity,False
onlineBackup,False
deviceProtection,False


In [ ]:
test.head()

,gender,seniorCitizen,dependents,tenure,phoneService,multipleLines,internetService,onlineSecurity,onlineBackup,deviceProtection,techSupport,streamingTV,streamingMovies,contract,paperlessBilling,paymentMethod,monthlyCharges,totalCharges
customerID,,,,,,,,,,,,,,,,,,
6295-OSINB,Male,0,False,72,True,True,Fiber optic,True,True,True,False,True,True,Two year,True,Electronic check,109.65,7880.25
1038-RQOST,Male,0,True,19,True,False,False,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,20.60,414.95
7613-LLQFO,Male,0,False,12,True,True,Fiber optic,False,False,False,False,True,False,Month-to-month,True,Electronic check,84.45,1059.55
4568-TTZRT,Male,0,False,9,True,False,False,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,False,Mailed check,20.40,181.8
9513-DXHDA,Male,0,False,27,True,False,DSL,True,False,True,True,True,True,One year,False,Electronic check,81.70,2212.55


In [ ]:
test.totalCharges = test.totalCharges.str.strip()

### Customer's potentially will need to be dropped because have no totalCharges.

In [ ]:
noTotal # given that these customer's total charges must be similar to those of their 'nearest neighbors'
# will simply apply the KNearestNeighbors imputer here

,gender,seniorCitizen,dependents,tenure,phoneService,multipleLines,internetService,onlineSecurity,onlineBackup,deviceProtection,techSupport,streamingTV,streamingMovies,contract,paperlessBilling,paymentMethod,monthlyCharges,totalCharges
customerID,,,,,,,,,,,,,,,,,,
7644-OMVMY,Male,0,True,0,True,False,False,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,19.85,
3213-VVOLG,Male,0,True,0,True,True,False,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,25.35,
2520-SGTTA,Female,0,True,0,True,False,False,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,False,Mailed check,20.00,


In [ ]:
totalCharges = test.pop('totalCharges')

### Dummy Variables

In [ ]:
totalCharges

,totalCharges
customerID,
6295-OSINB,7880.25
1038-RQOST,414.95
7613-LLQFO,1059.55
4568-TTZRT,181.8
9513-DXHDA,2212.55
...,...
7912-SYRQT,552.95
3557-HTYWR,3496.3
4816-JBHOV,93.7


In [ ]:
(test.isnull()).any()

,0
gender,False
seniorCitizen,False
dependents,False
tenure,False
phoneService,False
multipleLines,False
internetService,False
onlineSecurity,False
onlineBackup,False
deviceProtection,False


In [ ]:
test.apply(lambda x: '' in x)

,0
gender,False
seniorCitizen,False
dependents,False
tenure,False
phoneService,False
multipleLines,False
internetService,False
onlineSecurity,False
onlineBackup,False
deviceProtection,False


In [ ]:
pd.set_option('display.max_columns', 35)

In [ ]:
test = pd.get_dummies(test, dtype=int, prefix_sep='_', drop_first=True)

In [ ]:
test.shape

(2002, 28)

In [ ]:
test = test.join(totalCharges, how='inner')

### Imputing Values for totalCharges

In [ ]:
noTotal = test.query("totalCharges==''").index

In [ ]:
noTotal

Index(['7644-OMVMY', '3213-VVOLG', '2520-SGTTA'], dtype='object', name='customerID')

In [ ]:
noTotal = test.loc[noTotal]

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
total_ind = pd.concat([test.drop(noTotal.index, axis=0).assign(
    totalCharges = lambda x: x.totalCharges.astype(float)
), noTotal], axis=0).index

In [ ]:
noTotal = noTotal.assign(
    totalCharges = np.nan
)

In [ ]:
KNNImpute = KNNImputer()

In [ ]:
test = pd.DataFrame(KNNImpute.fit_transform((pd.concat([test.drop(noTotal.index, axis=0).assign(
    totalCharges = lambda x: x.totalCharges.astype(float)
), noTotal], axis=0))), columns=test.columns, index=total_ind)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2002 entries, 6295-OSINB to 2520-SGTTA
Data columns (total 29 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   seniorCitizen                          2002 non-null   float64
 1   dependents                             2002 non-null   float64
 2   tenure                                 2002 non-null   float64
 3   phoneService                           2002 non-null   float64
 4   paperlessBilling                       2002 non-null   float64
 5   monthlyCharges                         2002 non-null   float64
 6   gender_Male                            2002 non-null   float64
 7   multipleLines_True                     2002 non-null   float64
 8   multipleLines_No phone service         2002 non-null   float64
 9   internetService_DSL                    2002 non-null   float64
 10  internetService_Fiber optic            2002 non-null   float64

### Joining response back to test dataset

In [ ]:
test_response.shape

(2002,)

In [ ]:
test.shape[1]

29

In [ ]:
test = test.join(test_response, how='inner')

In [ ]:
test.to_excel('test_for_machine_learningdf.xlsx') # in case needed

### Ensuring both original dataset and test dataset have same columns

In [ ]:
test = pd.read_excel('/content/test_for_machine_learningdf.xlsx')

In [ ]:
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [ ]:
original.head()

,seniorCitizen,partner,dependents,tenure,phoneService,paperlessBilling,monthlyCharges,totalCharges,gender_Male,multipleLines_True,...,streamingTV_True,streamingTV_No internet service,streamingMovies_True,streamingMovies_No internet service,contract_One year,contract_Two year,paymentMethod_Credit card (automatic),paymentMethod_Electronic check,paymentMethod_Mailed check,churn
customerID,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,0,1,0,1,0,1,29.85,29.85,0,0,...,0,0,0,0,0,0,0,1,0,0
5575-GNVDE,0,0,0,34,1,0,56.95,1889.50,1,0,...,0,0,0,0,1,0,0,0,1,0
3668-QPYBK,0,0,0,2,1,1,53.85,108.15,1,0,...,0,0,0,0,0,0,0,0,1,1
7795-CFOCW,0,0,0,45,0,0,42.30,1840.75,1,0,...,0,0,0,0,1,0,0,0,0,0
9237-HQITU,0,0,0,2,1,1,70.70,151.65,0,0,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
test = test.assign(
    churn = lambda x: x.churn.astype(int)
)

In [ ]:
original.drop('partner', axis=1, inplace=True)

In [ ]:
original.shape

(5042, 30)

In [ ]:
test.shape

(2002, 31)

In [ ]:
original.columns.difference(test.columns)

Index([], dtype='object')

In [ ]:
test.columns.difference(original.columns)

Index(['customerID'], dtype='object')

In [ ]:
test.set_index('customerID', inplace=True)

In [ ]:
pd.set_option('display.max_columns', len(test.columns)+1)

In [ ]:
test = test[original.columns]

# Splitting the original dataset into training and testing dataframes

In [ ]:
X = original.drop('churn', axis=1)

In [ ]:
y = original.pop('churn')

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=0
)

## Creating Pipeline and Best Logistic Regression Model

In [ ]:
continuous = ['tenure', 'monthlyCharges', 'totalCharges']

In [ ]:
categorical = [col for col in X.columns if col not in continuous]

In [ ]:
only_numerical = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous),
        ('cat', 'passthrough', categorical)
    ]
)

In [ ]:
lr_pipeline = Pipeline([
    ('scaling', StandardScaler()),
    ('lr', LogisticRegression(class_weight='balanced', penalty='l2', random_state=0))
])

In [ ]:
param_grid = [
    {
        'lr__solver': ['lbfgs', 'newton-cg'],
        'lr__max_iter': [100, 1000],
        'lr__C': [0.01, 0.1, 1, 5, 10],
        'lr__fit_intercept': [False, True]
    }
]

In [ ]:
lr_grid = GridSearchCV(
    lr_pipeline, param_grid=param_grid, scoring='recall', cv=5
)

# Fitting Model

In [ ]:
lr_grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaling', StandardScaler()),
                                       ('lr',
                                        LogisticRegression(class_weight='balanced',
                                                           random_state=0))]),
             param_grid=[{'lr__C': [0.01, 0.1, 1, 5, 10],
                          'lr__fit_intercept': [False, True],
                          'lr__max_iter': [100, 1000],
                          'lr__solver': ['lbfgs', 'newton-cg']}],
             scoring='recall')

In [ ]:
lr_grid.best_params_

{'lr__C': 5,
 'lr__fit_intercept': False,
 'lr__max_iter': 100,
 'lr__solver': 'lbfgs'}

## Analyzing Results (Still Need Lower Threshold)

In [ ]:
class_names=['no churn', 'churn']

In [ ]:
classificationSummary(y_valid, lr_grid.predict(X_valid), class_names=class_names)

Confusion Matrix (Accuracy 0.7188)

         Prediction
  Actual no churn    churn
no churn      244      127
   churn       15      119


### COMMENTS: Interesting that the model performs better when the entire training set is fit to the pipeline using StandardScaler() rather than the ColumnTransfer() method which only scaled the continuous features. Will run on new unseen (test data) to ensure model performs well and 'good recall performance' isn't due to random chance or overfitting.

### Recalibrating optimal threshold for high TPR/low FNR

In [ ]:
df_valid = pd.DataFrame({
    'predicted': lr_grid.predict(X_valid),
    'actual': y_valid,
    'prob_churn': lr_grid.predict_proba(X_valid)[:,1]
})

In [ ]:
def classSummary(data, class_summ, cutoff, class_names):
  predicted = [1 if p > cutoff else 0 for p in data.prob_churn]

  class_summ(data.actual, predicted, class_names=class_names)

In [ ]:
#cutoff = 0.45 on validation

classSummary(df_valid, classificationSummary, 0.45, class_names)

Confusion Matrix (Accuracy 0.6931)

         Prediction
  Actual no churn    churn
no churn      224      147
   churn        8      126


# Peformance on Test Data (Threshold 0.5)

In [ ]:
test.head()

,seniorCitizen,dependents,tenure,phoneService,paperlessBilling,monthlyCharges,totalCharges,gender_Male,multipleLines_True,multipleLines_No phone service,internetService_DSL,internetService_Fiber optic,onlineSecurity_True,onlineSecurity_No internet service,onlineBackup_True,onlineBackup_No internet service,deviceProtection_True,deviceProtection_No internet service,techSupport_True,techSupport_No internet service,streamingTV_True,streamingTV_No internet service,streamingMovies_True,streamingMovies_No internet service,contract_One year,contract_Two year,paymentMethod_Credit card (automatic),paymentMethod_Electronic check,paymentMethod_Mailed check,churn
customerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6295-OSINB,0,0,72,1,1,109.65,7880.25,1,1,0,0,1,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0
1038-RQOST,0,1,19,1,0,20.60,414.95,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0
7613-LLQFO,0,0,12,1,1,84.45,1059.55,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
4568-TTZRT,0,0,9,1,0,20.40,181.80,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0
9513-DXHDA,0,0,27,1,0,81.70,2212.55,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1,0,0


In [ ]:
test_X = test.drop('churn', axis=1)

In [ ]:
test_y = test.pop('churn')

In [ ]:
classificationSummary(test_y, lr_grid.predict(test_X), class_names=class_names) # with a threshold of 0.5

Confusion Matrix (Accuracy 0.6853)

         Prediction
  Actual no churn    churn
no churn      908      561
   churn       69      464


 - **Accuracy**: 69%
 - **Misclassification Rate**: 31%
 - **True Positive Rate**: 87%
 - **False Negative Rate**: 13%

In [ ]:
df = pd.DataFrame({
    'predicted': lr_grid.predict(test_X),
    'actual': test_y,
    'prob_churn': lr_grid.predict_proba(test_X)[:,1]
}).sort_values(by='prob_churn', ascending=False)

In [ ]:
classSummary(df, classificationSummary, 0.45, class_names)

Confusion Matrix (Accuracy 0.6533)

         Prediction
  Actual no churn    churn
no churn      822      647
   churn       47      486


**Total Customers in Test Data**: 2002
 - **Accuracy Rate**: 65.33%
 - **Misclassification Rate**: 34.67%
 - **True Positive Rate**: 91.18%
 - **False Negative Rate**: 8.82%

In [ ]:
# comparing to validation

classSummary(df_valid, classificationSummary, 0.45, class_names)

Confusion Matrix (Accuracy 0.6931)

         Prediction
  Actual no churn    churn
no churn      224      147
   churn        8      126


 **Total Customers in Validation Data**: 505
 - **Accuracy Rate**: 69.11%
 - **Misclassification Rate**: 30.89%
 - **True Positive Rate**: 94.03%
 - **False Negative Rate**: 5.97%

GREAT!

**NOTE**: Fixing the Fiber Optic Issue would, and theoretically keeping those customers who leave, would see around a 17% decrease in churn rate.